In [1]:
!pip install geopandas matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 79.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 73.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 32.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [geopandas]15 [geopandas]]


In [2]:
import geopandas
import numpy
import shapely

def generate_3d_climb_profile(runways: geopandas.GeoDataFrame, config: dict) -> shapely.Polygon:
    """Generates a 3D climb profile polygon aligned with the runway direction."""
    runway = runways.geometry.iloc[0]
    coords = numpy.array(runway.coords)
    runway_length = runway.length  # meters
    dx = coords[-1][0] - coords[0][0]
    dy = coords[-1][1] - coords[0][1]
    heading = numpy.arctan2(dy, dx)  # radians

    # Convert climb rate to altitude profile
    distance_beyond_runway = 30_000
    climb_rate_ms = config["climb_rate"] / 196.85  # ft/min → m/s
    distance_m = numpy.linspace(runway_length, runway_length + distance_beyond_runway, 100)

    # Calculate altitude profile
    altitude = numpy.zeros_like(distance_m)
    mask_climb = distance_m > runway_length
    time_to_climb = (distance_m[mask_climb] - runway_length) / 75  # 75 m/s speed
    altitude[mask_climb] = climb_rate_ms * time_to_climb

    # Calculate width expansion (15° divergence per ICAO)
    initial_width = 300  # meters (primary surface)
    width = numpy.where(
        distance_m <= runway_length,
        initial_width,
        initial_width + 2 * distance_m * numpy.tan(numpy.radians(7.5))  # 15° total divergence
    )

    # Generate left and right boundaries
    x_center = coords[0][0] + (distance_m * numpy.cos(heading))
    y_center = coords[0][1] + (distance_m * numpy.sin(heading))
    
    # Perpendicular vectors for width expansion
    perp_heading = heading + numpy.pi / 2
    x_left = x_center + (width / 2 * numpy.cos(perp_heading))
    y_left = y_center + (width / 2 * numpy.sin(perp_heading))
    x_right = x_center - (width / 2 * numpy.cos(perp_heading))
    y_right = y_center - (width / 2 * numpy.sin(perp_heading))

    # Create 3D polygons
    top_points = list(zip(x_left, y_left, altitude))
    bottom_points = list(zip(x_right[::-1], y_right[::-1], altitude[::-1]))
    polygon_points = top_points + bottom_points
    
    # Ensure closure
    if polygon_points[0] != polygon_points[-1]:
        polygon_points.append(polygon_points[0])
    
    # Create 3D polygon (combine x, y, z)
    return shapely.MultiPolygon([shapely.Polygon(polygon_points)])

In [3]:
import json

def save_polygon(polygon, filename):
    # Create GeoJSON feature dictionary
    feature = {
        "type": "Feature",
        # Convert Shapely polygon to GeoJSON geometry
        "geometry": shapely.geometry.mapping(polygon),
        "properties": {},
        "crs": {
            "type": "name",
            "properties": {
                "name": "EPSG:27040"
            }
        }
    }
    with open(filename, "w") as output:
        json.dump(feature, output, indent=2)

In [4]:
# Define climb rates for different aircraft Feet Per Minute (FPM)
MIN_CLIMB_RATE = 1000  * 0.3048     # Regulatory minimum (e.g., engine-out scenario)
AVG_CLIMB_RATE = 2000 * 0.3048     # Typical commercial jet (A320/B737)
MAX_CLIMB_RATE = 4000 * 0.3048     # High-performance (e.g., lightly loaded A321)
# Load runway shapefile
runways = geopandas.read_file("./data/runways.geojson")

In [8]:
# climb profile for nominal flight
min_climb_profile_config = dict(
    climb_rate=MIN_CLIMB_RATE,
    climb_gradient=0.024,   # 2.4% i.e., twin-engine, with engine failure
)
# Generate polygons by profile type
save_polygon(generate_3d_climb_profile(runways, min_climb_profile_config), "./data/min_climb_profile.geojson")

In [9]:
# climb profile for nominal flight
nominal_config = dict(
    climb_rate=AVG_CLIMB_RATE,
    climb_gradient=0.012, # 1.2% i.e., twin-engine, without engine failure
)
# Generate polygons by profile type
save_polygon(generate_3d_climb_profile(runways, nominal_config), "./data/avg_climb_profile.geojson")

In [10]:
# climb profile for nominal flight
max_climb_profile_config = dict(
    climb_rate=MAX_CLIMB_RATE,
    climb_gradient=0.008, # 0.8% i.e., multi-engine, without engine failure
)
# Generate polygons by profile type
save_polygon(generate_3d_climb_profile(runways, max_climb_profile_config), "./data/max_climb_profile.geojson")